# Assignment Solution 1: Analysis of Power Usage Data

#### Instructions 

We will use a dataset from one smartmeter to analyze the energy consumption pattern of a house. Analyze the electricity usage pattern and see what conclusion you can draw about the resident?

Note:
1. You need to pay attention to missing data;
2. calculate some aggregate values of energy usage and observe different type of trends (e.g. pattern in a day, pattern in a week, pattern in a year, etc);
3. Use Spark to do your calculations, then use dataframes to draw some plots. Describe each plot briefly and draw some conclusions;
4. You only need to use the simple Spark transformations and actions covered in class, no need to use machine learning methods yet.

#### Description of the Dataset
Source: https://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+consumption#

This archive contains 2075259 measurements gathered between December 2006 and November 2010 (47 months). 

Notes: 

1.(global_active_power*1000/60 - sub_metering_1 - sub_metering_2 - sub_metering_3) represents the active energy consumed every minute (in watt hour) in the household by electrical equipment not measured in sub-meterings 1, 2 and 3. 

2.The dataset contains some missing values in the measurements (nearly 1,25% of the rows). All calendar timestamps are present in the dataset but for some timestamps, the measurement values are missing: a missing value is represented by the absence of value between two consecutive semi-colon attribute separators. For instance, the dataset shows missing values on April 28, 2007.


Attribute Information:

1.date: Date in format dd/mm/yyyy 

2.time: time in format hh:mm:ss 

3.global_active_power: household global minute-averaged active power (in kilowatt) 

4.global_reactive_power: household global minute-averaged reactive power (in kilowatt) 

5.voltage: minute-averaged voltage (in volt) 

6.global_intensity: household global minute-averaged current intensity (in ampere) 

7.sub_metering_1: energy sub-metering No. 1 (in watt-hour of active energy). It corresponds to the kitchen, containing mainly a dishwasher, an oven and a microwave (hot plates are not electric but gas powered). 

8.sub_metering_2: energy sub-metering No. 2 (in watt-hour of active energy). It corresponds to the laundry room, containing a washing-machine, a tumble-drier, a refrigerator and a light. 

9.sub_metering_3: energy sub-metering No. 3 (in watt-hour of active energy). It corresponds to an electric water-heater and an air-conditioner.

#### Load the data into an RDD

In [5]:
#read txt file, gzipped files will be auto-unzipped
myRDD = sc.textFile("/mnt/mlonspark/household_power_consumption.txt.gz")
print myRDD.count()
myRDD.take(5)

#### Load Data to Spark Dataframe and describe data.

In [7]:
from pyspark.sql.types import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

tuples = myRDD.map(lambda x:x.split(";")).map(lambda x:[x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8]])
fields=["dt", "time", "global_active_power", "global_reactive_power", "voltage", "global_intensity", "sub_metering_1", "sub_metering_2", "sub_metering_3"]
origDF=sqlContext.createDataFrame(tuples, fields)
origDF.show(5)

#### Data Pre-processing & Transformation

In [9]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

func =  udf (lambda x: datetime.strptime(x, '%d/%m/%Y'), DateType())
df = origDF.withColumn('date', func(col('dt')))
df.printSchema()
df.show(5)
df.createOrReplaceTempView("temp")

#### Active Power Consumption - Yearly Trend

In [11]:
active_power_consumption_per_year_sum=spark.sql("select year(date) as year, round(sum(global_active_power)) as active_power from temp group by year order by year")
display(active_power_consumption_per_year_sum)

**Analysis : **
 
 
 1. As the dataset covers for only 1 month (Decemeber) in 2006, hence the power consumption shows very less compared to other years.
 2. Similarly, for 2010, the dataset exists for only 11 months (until November), hence power consumption is slightly less compared to other year.
 3. Excluding 2006 and 2010, for all other years, power consumption remanins stable.

In [13]:
import pyspark.sql.functions as F
def myConcat(*cols): return F.concat(*[F.coalesce(c, F.lit("*")) for c in cols])
monthdf=dfyear.withColumn("yearmonth", myConcat("year", "month")).drop("year").drop("month") 
monthdf.printSchema()

#### Active Power Consumption - Monthly Trend in Each Year

In [15]:
import pyspark.sql.functions as F
def myConcat(*cols): return F.concat(*[F.coalesce(c, F.lit("*")) for c in cols])

yrmonthDF=spark.sql("select year(date) as year, month(date) as month, round(sum(global_active_power)) as active_power from temp group by year,month order by year,month")
yrmonthDF.show(5)
active_power_consumption_per_year_month=yrmonthDF.withColumn("yearmonth", myConcat("year", "month")).drop("year").drop("month") 
active_power_consumption_per_year_month.printSchema()

In [16]:
display(active_power_consumption_per_year_month)

**Analysis**
 
 
 1. <TBD>

#### Active Power Consumption - By Month

In [19]:
active_power_consumption_per_month_sum=spark.sql("select month(date) as month, round(sum(global_active_power)) as active_power from temp group by month order by month")
display(active_power_consumption_per_month_sum)

**Analysis**
 
 
 1. <TBD>

#### Active Power Consumption - Weekly Trend

In [22]:
from pyspark.sql.functions import date_format
weekday=spark.sql("select date_format(date,'E') as weekday, round(sum(global_active_power)) as active_power from temp group by weekday order by active_power desc ")
weekday.show(10)
display(weekday)

**Analysis**
 
 
 1. <TBD>

#### Active Power Consumption - Daily Trend

In [25]:
active_power_consumption_per_day_sum=spark.sql("select date, round(sum(global_active_power)) as active_power from temp group by date order by date")
display(active_power_consumption_per_day_sum)

**Analysis**
 
 
 1. <TBD>